In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from math import sin, cos, sqrt, atan2, radians
from sklearn.cluster import DBSCAN,KMeans
import random as rd
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

In [2]:
# Read first 2500000 rows of train file
df=pd.read_csv('new_train.csv',nrows=4000000)
df=df[df['dropoff_latitude'].notnull()]
del df['Unnamed: 0']
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2010-03-28 00:13:01.0000002,11.3,2010-03-28 00:13:01 UTC,-73.992285,40.727372,-73.955689,40.773357,1
1,2009-12-29 09:21:00.00000037,28.1,2009-12-29 09:21:00 UTC,-73.984918,40.758758,-73.873383,40.774368,1
2,2010-02-13 12:34:29.0000001,3.3,2010-02-13 12:34:29 UTC,-73.968866,40.754251,-73.974596,40.751740,1
3,2012-08-03 14:27:00.000000205,7.7,2012-08-03 14:27:00 UTC,-73.976125,40.760132,-73.984120,40.740300,2
4,2015-05-19 22:33:02.0000009,18.0,2015-05-19 22:33:02 UTC,-73.977867,40.749809,-73.948914,40.808022,1


In [3]:
# Read test file for rest of the preprocessing
dftest=pd.read_csv('test.csv')
dftest.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


In [4]:
# Split pickup_datetime into date and time
df['Date']=df.pickup_datetime.str.slice(start=0,stop=10)
df['Time']=df.pickup_datetime.str.slice(start=11,stop=19)
df.head()

df['Date']=pd.to_datetime(arg=df.Date)
df['month']=pd.DatetimeIndex(df.Date).month
df['year']=pd.DatetimeIndex(df.Date).year
df['weekday']=pd.DatetimeIndex(df.Date).weekday
df['hour']=pd.DatetimeIndex(df.key).hour
df['week_of_month']=pd.DatetimeIndex(df.key).day%7
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,month,year,weekday,hour,week_of_month
0,2010-03-28 00:13:01.0000002,11.3,2010-03-28 00:13:01 UTC,-73.992285,40.727372,-73.955689,40.773357,1,2010-03-28,00:13:01,3,2010,6,0,0
1,2009-12-29 09:21:00.00000037,28.1,2009-12-29 09:21:00 UTC,-73.984918,40.758758,-73.873383,40.774368,1,2009-12-29,09:21:00,12,2009,1,9,1
2,2010-02-13 12:34:29.0000001,3.3,2010-02-13 12:34:29 UTC,-73.968866,40.754251,-73.974596,40.751740,1,2010-02-13,12:34:29,2,2010,5,12,6
3,2012-08-03 14:27:00.000000205,7.7,2012-08-03 14:27:00 UTC,-73.976125,40.760132,-73.984120,40.740300,2,2012-08-03,14:27:00,8,2012,4,14,3
4,2015-05-19 22:33:02.0000009,18.0,2015-05-19 22:33:02 UTC,-73.977867,40.749809,-73.948914,40.808022,1,2015-05-19,22:33:02,5,2015,1,22,5


In [5]:
# For test file
# Split pickup_datetime into date and time
dftest['Date']=dftest.pickup_datetime.str.slice(start=0,stop=10)
dftest['Time']=dftest.pickup_datetime.str.slice(start=11,stop=19)
dftest.head()

dftest['Date']=pd.to_datetime(arg=dftest.Date)
dftest['month']=pd.DatetimeIndex(dftest.Date).month
dftest['year']=pd.DatetimeIndex(dftest.Date).year
dftest['weekday']=pd.DatetimeIndex(dftest.Date).weekday
dftest['hour']=pd.DatetimeIndex(dftest.key).hour
dftest['week_of_month']=pd.DatetimeIndex(dftest.key).day%7
dftest.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,month,year,weekday,hour,week_of_month
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1,2015-01-27,13:08:24,1,2015,1,13,6
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1,2015-01-27,13:08:24,1,2015,1,13,6
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1,2011-10-08,11:53:44,10,2011,5,11,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1,2012-12-01,21:12:12,12,2012,5,21,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1,2012-12-01,21:12:12,12,2012,5,21,1


In [6]:
def preprocess_hour(val):
    if val>=17:
        return 1
    elif val>7 and val<17:
        return 2
    elif val==0:
        return 3
    else:
        return 4

df['processed_hour_num_of_records']=df['hour'].apply(preprocess_hour)
dftest['processed_hour_num_of_records']=dftest['hour'].apply(preprocess_hour)
dftest.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,month,year,weekday,hour,week_of_month,processed_hour_num_of_records
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1,2015-01-27,13:08:24,1,2015,1,13,6,2
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1,2015-01-27,13:08:24,1,2015,1,13,6,2
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1,2011-10-08,11:53:44,10,2011,5,11,1,2
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1,2012-12-01,21:12:12,12,2012,5,21,1,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1,2012-12-01,21:12:12,12,2012,5,21,1,1


In [7]:
di=df[['hour','fare_amount']].groupby([ "hour"]).median().to_dict()
def process_hour_fare(val):
    return di['fare_amount'][val]

df['processed_hour_fare']=df['hour'].apply(process_hour_fare)
dftest['processed_hour_fare']=dftest['hour'].apply(process_hour_fare)
dftest.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,month,year,weekday,hour,week_of_month,processed_hour_num_of_records,processed_hour_fare
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1,2015-01-27,13:08:24,1,2015,1,13,6,2,8.5
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1,2015-01-27,13:08:24,1,2015,1,13,6,2,8.5
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1,2011-10-08,11:53:44,10,2011,5,11,1,2,8.5
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1,2012-12-01,21:12:12,12,2012,5,21,1,1,8.5
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1,2012-12-01,21:12:12,12,2012,5,21,1,1,8.5


In [8]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,month,year,weekday,hour,week_of_month,processed_hour_num_of_records,processed_hour_fare
0,2010-03-28 00:13:01.0000002,11.3,2010-03-28 00:13:01 UTC,-73.992285,40.727372,-73.955689,40.773357,1,2010-03-28,00:13:01,3,2010,6,0,0,3,8.9
1,2009-12-29 09:21:00.00000037,28.1,2009-12-29 09:21:00 UTC,-73.984918,40.758758,-73.873383,40.774368,1,2009-12-29,09:21:00,12,2009,1,9,1,2,8.5
2,2010-02-13 12:34:29.0000001,3.3,2010-02-13 12:34:29 UTC,-73.968866,40.754251,-73.974596,40.751740,1,2010-02-13,12:34:29,2,2010,5,12,6,2,8.1
3,2012-08-03 14:27:00.000000205,7.7,2012-08-03 14:27:00 UTC,-73.976125,40.760132,-73.984120,40.740300,2,2012-08-03,14:27:00,8,2012,4,14,3,2,8.5
4,2015-05-19 22:33:02.0000009,18.0,2015-05-19 22:33:02 UTC,-73.977867,40.749809,-73.948914,40.808022,1,2015-05-19,22:33:02,5,2015,1,22,5,1,8.5


In [9]:
def distance(coord1lat, coord1lon, coord2lat, coord2lon):
    R = 6373.0
    lat1 = radians(float(coord1lat))
    lon1 = radians(float(coord1lon))
    lat2 = radians(float(coord2lat))
    lon2 = radians(float(coord2lon))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

def manhattan(coord1lat, coord1lon, coord2lat, coord2lon):
    d1=distance(coord1lat, coord2lon, coord2lat, coord2lon)
    d2=distance(coord1lat, coord1lon, coord1lat, coord2lon)
    
    return d1+d2

In [10]:
df['Distance']=df[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']].apply(lambda x: distance(x[0], x[1], x[2], x[3]), axis=1)
df['Manhattan_Distance']=df[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']].apply(lambda x: manhattan(x[0], x[1], x[2], x[3]), axis=1)
dftest['Distance']=dftest[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']].apply(lambda x: distance(x[0], x[1], x[2], x[3]), axis=1)
dftest['Manhattan_Distance']=dftest[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']].apply(lambda x: manhattan(x[0], x[1], x[2], x[3]), axis=1)

max(dftest['Manhattan_Distance'])

141.11721758277946

In [11]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,month,year,weekday,hour,week_of_month,processed_hour_num_of_records,processed_hour_fare,Distance,Manhattan_Distance
0,2010-03-28 00:13:01.0000002,11.3,2010-03-28 00:13:01 UTC,-73.992285,40.727372,-73.955689,40.773357,1,2010-03-28,00:13:01,3,2010,6,0,0,3,8.9,5.972559,8.199672
1,2009-12-29 09:21:00.00000037,28.1,2009-12-29 09:21:00 UTC,-73.984918,40.758758,-73.873383,40.774368,1,2009-12-29,09:21:00,12,2009,1,9,1,2,8.5,9.555103,11.133425
2,2010-02-13 12:34:29.0000001,3.3,2010-02-13 12:34:29 UTC,-73.968866,40.754251,-73.974596,40.751740,1,2010-02-13,12:34:29,2,2010,5,12,6,2,8.1,0.557775,0.762099
3,2012-08-03 14:27:00.000000205,7.7,2012-08-03 14:27:00 UTC,-73.976125,40.760132,-73.984120,40.740300,2,2012-08-03,14:27:00,8,2012,4,14,3,2,8.5,2.306489,2.879497
4,2015-05-19 22:33:02.0000009,18.0,2015-05-19 22:33:02 UTC,-73.977867,40.749809,-73.948914,40.808022,1,2015-05-19,22:33:02,5,2015,1,22,5,1,8.5,6.918961,8.914686


In [12]:
# Clustering using DBSCAN
#clus=DBSCAN(eps=0.0002353, min_samples=4,algorithm='ball_tree',metric='haversine')
clus=KMeans(n_clusters=25)
df['Pickup_clusters']=clus.fit_predict(df[['pickup_latitude','pickup_longitude']].values)
dftest['Pickup_clusters']=clus.predict(dftest[['pickup_latitude','pickup_longitude']].values)

In [13]:
# df.plot.scatter(y='pickup_latitude',x='pickup_longitude')
# plt.show()
dftest.loc[dftest['Distance']==0,:]

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,month,year,weekday,hour,week_of_month,processed_hour_num_of_records,processed_hour_fare,Distance,Manhattan_Distance,Pickup_clusters
121,2014-06-14 13:39:00.000000191,2014-06-14 13:39:00 UTC,-73.980590,40.747682,-73.980590,40.747682,1,2014-06-14,13:39:00,6,2014,5,13,0,2,8.5,0.0,0.0,22
279,2010-09-05 22:31:32.0000002,2010-09-05 22:31:32 UTC,-74.047394,40.785789,-74.047394,40.785789,1,2010-09-05,22:31:32,9,2010,6,22,5,1,8.5,0.0,0.0,13
318,2009-06-10 16:55:00.000000131,2009-06-10 16:55:00 UTC,-73.985862,40.744027,-73.985862,40.744027,1,2009-06-10,16:55:00,6,2009,2,16,3,2,8.1,0.0,0.0,4
417,2011-06-24 12:03:00.000000145,2011-06-24 12:03:00 UTC,-73.964615,40.777620,-73.964615,40.777620,1,2011-06-24,12:03:00,6,2011,4,12,3,2,8.1,0.0,0.0,6
423,2011-06-24 12:03:00.00000086,2011-06-24 12:03:00 UTC,-73.955065,40.771505,-73.955065,40.771505,1,2011-06-24,12:03:00,6,2011,4,12,3,2,8.1,0.0,0.0,6
434,2011-06-24 12:03:00.000000173,2011-06-24 12:03:00 UTC,-73.960738,40.775717,-73.960738,40.775717,1,2011-06-24,12:03:00,6,2011,4,12,3,2,8.1,0.0,0.0,6
451,2011-06-24 12:03:00.00000047,2011-06-24 12:03:00 UTC,-73.967895,40.765477,-73.967895,40.765477,1,2011-06-24,12:03:00,6,2011,4,12,3,2,8.1,0.0,0.0,0
491,2011-06-24 12:03:00.00000076,2011-06-24 12:03:00 UTC,-73.830820,40.759288,-73.830820,40.759288,1,2011-06-24,12:03:00,6,2011,4,12,3,2,8.1,0.0,0.0,19
498,2011-06-24 12:03:00.00000051,2011-06-24 12:03:00 UTC,-73.945490,40.752820,-73.945490,40.752820,1,2011-06-24,12:03:00,6,2011,4,12,3,2,8.1,0.0,0.0,16
666,2012-11-11 20:35:40.0000002,2012-11-11 20:35:40 UTC,-73.970087,40.755266,-73.970087,40.755266,1,2012-11-11,20:35:40,11,2012,6,20,4,1,8.1,0.0,0.0,22


In [14]:
#clus=DBSCAN(eps=0.0002353, min_samples=4,algorithm='ball_tree',metric='haversine')
clus=KMeans(n_clusters=25)
df['Dropoff_clusters']=clus.fit_predict(df[['dropoff_latitude','dropoff_longitude']].values)
dftest['Dropoff_clusters']=clus.predict(dftest[['dropoff_latitude','dropoff_longitude']].values)

In [15]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,...,year,weekday,hour,week_of_month,processed_hour_num_of_records,processed_hour_fare,Distance,Manhattan_Distance,Pickup_clusters,Dropoff_clusters
0,2010-03-28 00:13:01.0000002,11.3,2010-03-28 00:13:01 UTC,-73.992285,40.727372,-73.955689,40.773357,1,2010-03-28,00:13:01,...,2010,6,0,0,3,8.9,5.972559,8.199672,17,14
1,2009-12-29 09:21:00.00000037,28.1,2009-12-29 09:21:00 UTC,-73.984918,40.758758,-73.873383,40.774368,1,2009-12-29,09:21:00,...,2009,1,9,1,2,8.5,9.555103,11.133425,9,8
2,2010-02-13 12:34:29.0000001,3.3,2010-02-13 12:34:29 UTC,-73.968866,40.754251,-73.974596,40.751740,1,2010-02-13,12:34:29,...,2010,5,12,6,2,8.1,0.557775,0.762099,0,0
3,2012-08-03 14:27:00.000000205,7.7,2012-08-03 14:27:00 UTC,-73.976125,40.760132,-73.984120,40.740300,2,2012-08-03,14:27:00,...,2012,4,14,3,2,8.5,2.306489,2.879497,22,4
4,2015-05-19 22:33:02.0000009,18.0,2015-05-19 22:33:02 UTC,-73.977867,40.749809,-73.948914,40.808022,1,2015-05-19,22:33:02,...,2015,1,22,5,1,8.5,6.918961,8.914686,22,3


In [16]:
di=df[['Pickup_clusters','fare_amount']].groupby(["Pickup_clusters"]).median().to_dict()
def process_pickup_fare(val):
    return di['fare_amount'][val]

df['processed_pickup_fare']=df['Pickup_clusters'].apply(process_pickup_fare)
dftest['processed_pickup_fare']=dftest['Pickup_clusters'].apply(process_pickup_fare)

In [17]:
di=df[['Dropoff_clusters','fare_amount']].groupby(["Dropoff_clusters"]).median().to_dict()
def process_dropoff_fare(val):
    return di['fare_amount'][val]

df['processed_dropoff_fare']=df['Dropoff_clusters'].apply(process_dropoff_fare)
dftest['processed_dropoff_fare']=dftest['Dropoff_clusters'].apply(process_dropoff_fare)

KeyError: (58, 17)

In [19]:
di=df[['Pickup_clusters','Dropoff_clusters','fare_amount']].groupby(['Pickup_clusters',"Dropoff_clusters"]).median().to_dict()
def process_route_fare(val):
    return di['fare_amount'][val]

df['processed_route_fare']=df[['Pickup_clusters','Dropoff_clusters']].apply(lambda x:process_route_fare((x.Pickup_clusters,x.Dropoff_clusters)),axis=1)
dftest['processed_route_fare']=dftest[['Pickup_clusters','Dropoff_clusters']].apply(lambda x:process_route_fare((x.Pickup_clusters,x.Dropoff_clusters)),axis=1)

In [20]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,...,week_of_month,processed_hour_num_of_records,processed_hour_fare,Distance,Manhattan_Distance,Pickup_clusters,Dropoff_clusters,processed_pickup_fare,processed_dropoff_fare,processed_route_fare
0,2010-03-28 00:13:01.0000002,11.3,2010-03-28 00:13:01 UTC,-73.992285,40.727372,-73.955689,40.773357,1,2010-03-28,00:13:01,...,0,3,8.9,5.972559,8.199672,17,14,8.9,7.7,14.1
1,2009-12-29 09:21:00.00000037,28.1,2009-12-29 09:21:00 UTC,-73.984918,40.758758,-73.873383,40.774368,1,2009-12-29,09:21:00,...,1,2,8.5,9.555103,11.133425,9,8,8.1,28.9,32.5
2,2010-02-13 12:34:29.0000001,3.3,2010-02-13 12:34:29 UTC,-73.968866,40.754251,-73.974596,40.751740,1,2010-02-13,12:34:29,...,6,2,8.1,0.557775,0.762099,0,0,7.5,7.7,5.5
3,2012-08-03 14:27:00.000000205,7.7,2012-08-03 14:27:00 UTC,-73.976125,40.760132,-73.984120,40.740300,2,2012-08-03,14:27:00,...,3,2,8.5,2.306489,2.879497,22,4,7.7,7.5,6.1
4,2015-05-19 22:33:02.0000009,18.0,2015-05-19 22:33:02 UTC,-73.977867,40.749809,-73.948914,40.808022,1,2015-05-19,22:33:02,...,5,1,8.5,6.918961,8.914686,22,3,7.7,10.0,14.5


In [21]:
di=df[['Pickup_clusters','Dropoff_clusters','fare_amount']].groupby(['Pickup_clusters',"Dropoff_clusters"]).count().to_dict()
def process_route_popularity(val):
    return di['fare_amount'][val]

df['processed_route_rank']=df[['Pickup_clusters','Dropoff_clusters']].apply(lambda x:process_route_popularity((x.Pickup_clusters,x.Dropoff_clusters)),axis=1)
dftest['processed_route_rank']=dftest[['Pickup_clusters','Dropoff_clusters']].apply(lambda x:process_route_popularity((x.Pickup_clusters,x.Dropoff_clusters)),axis=1)

In [22]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,...,processed_hour_num_of_records,processed_hour_fare,Distance,Manhattan_Distance,Pickup_clusters,Dropoff_clusters,processed_pickup_fare,processed_dropoff_fare,processed_route_fare,processed_route_rank
0,2010-03-28 00:13:01.0000002,11.3,2010-03-28 00:13:01 UTC,-73.992285,40.727372,-73.955689,40.773357,1,2010-03-28,00:13:01,...,3,8.9,5.972559,8.199672,17,14,8.9,7.7,14.1,16388
1,2009-12-29 09:21:00.00000037,28.1,2009-12-29 09:21:00 UTC,-73.984918,40.758758,-73.873383,40.774368,1,2009-12-29,09:21:00,...,2,8.5,9.555103,11.133425,9,8,8.1,28.9,32.5,8765
2,2010-02-13 12:34:29.0000001,3.3,2010-02-13 12:34:29 UTC,-73.968866,40.754251,-73.974596,40.751740,1,2010-02-13,12:34:29,...,2,8.1,0.557775,0.762099,0,0,7.5,7.7,5.5,92983
3,2012-08-03 14:27:00.000000205,7.7,2012-08-03 14:27:00 UTC,-73.976125,40.760132,-73.984120,40.740300,2,2012-08-03,14:27:00,...,2,8.5,2.306489,2.879497,22,4,7.7,7.5,6.1,88510
4,2015-05-19 22:33:02.0000009,18.0,2015-05-19 22:33:02 UTC,-73.977867,40.749809,-73.948914,40.808022,1,2015-05-19,22:33:02,...,1,8.5,6.918961,8.914686,22,3,7.7,10.0,14.5,5795


In [23]:
di=df[['Pickup_clusters','Dropoff_clusters','passenger_count','fare_amount']].groupby(['Pickup_clusters',"Dropoff_clusters",'passenger_count']).median().to_dict()
def process_route_people_fare(val):
    return di['fare_amount'][val]

df['processed_route_people_fare']=df[['Pickup_clusters','Dropoff_clusters','passenger_count']].apply(lambda x:process_route_people_fare((x.Pickup_clusters,x.Dropoff_clusters,x.passenger_count)),axis=1)
dftest['processed_route_people_fare']=dftest[['Pickup_clusters','Dropoff_clusters','passenger_count']].apply(lambda x:process_route_people_fare((x.Pickup_clusters,x.Dropoff_clusters,x.passenger_count)),axis=1)
dftest.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,month,...,processed_hour_fare,Distance,Manhattan_Distance,Pickup_clusters,Dropoff_clusters,processed_pickup_fare,processed_dropoff_fare,processed_route_fare,processed_route_rank,processed_route_people_fare
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1,2015-01-27,13:08:24,1,...,8.5,2.323989,2.904495,0,4,7.5,7.5,8.5,56004,8.5
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1,2015-01-27,13:08:24,1,...,8.5,2.426114,3.217929,17,17,8.9,7.5,7.5,46827,7.3
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1,2011-10-08,11:53:44,10,...,8.5,0.618822,0.811441,22,4,7.7,7.5,6.1,88510,6.1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1,2012-12-01,21:12:12,12,...,8.5,1.961648,2.581242,9,4,8.1,7.5,8.5,51438,8.5
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1,2012-12-01,21:12:12,12,...,8.5,5.388992,6.940452,11,4,8.0,7.5,13.3,12662,13.3


In [29]:
df.processed_hour_num_of_records

0          3
1          2
2          2
3          2
4          1
5          1
6          4
7          1
8          4
9          4
10         1
11         2
12         2
13         1
14         1
15         2
16         2
17         2
18         2
19         2
20         2
21         1
22         2
23         1
24         2
25         2
26         1
27         1
28         2
29         2
          ..
3999970    2
3999971    2
3999972    3
3999973    2
3999974    1
3999975    2
3999976    1
3999977    2
3999978    4
3999979    4
3999980    2
3999981    4
3999982    1
3999983    2
3999984    2
3999985    2
3999986    1
3999987    1
3999988    1
3999989    2
3999990    3
3999991    1
3999992    1
3999993    1
3999994    1
3999995    4
3999996    1
3999997    2
3999998    2
3999999    2
Name: processed_hour_num_of_records, Length: 4000000, dtype: int64

In [32]:
di=df[['Pickup_clusters','Dropoff_clusters','processed_hour_num_of_records','passenger_count','fare_amount']].groupby(['Pickup_clusters',"Dropoff_clusters",'processed_hour_num_of_records','passenger_count']).median().to_dict()
def process_route_hour_people_fare(val):
    return di['fare_amount'][val]

df['processed_route_hour_people_fare']=df[['Pickup_clusters','Dropoff_clusters','processed_hour_num_of_records','passenger_count']].apply(lambda x:process_route_hour_people_fare((x.Pickup_clusters,x.Dropoff_clusters,x.processed_hour_num_of_records,x.passenger_count)),axis=1)
dftest['processed_route_hour_people_fare']=dftest[['Pickup_clusters','Dropoff_clusters','processed_hour_num_of_records','passenger_count']].apply(lambda x:process_route_hour_people_fare((x.Pickup_clusters,x.Dropoff_clusters,x.processed_hour_num_of_records,x.passenger_count)),axis=1)
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Date,Time,...,Distance,Manhattan_Distance,Pickup_clusters,Dropoff_clusters,processed_pickup_fare,processed_dropoff_fare,processed_route_fare,processed_route_rank,processed_route_people_fare,processed_route_hour_people_fare
0,2010-03-28 00:13:01.0000002,11.3,2010-03-28 00:13:01 UTC,-73.992285,40.727372,-73.955689,40.773357,1,2010-03-28,00:13:01,...,5.972559,8.199672,17,14,8.9,7.7,14.1,16388,14.10,13.70
1,2009-12-29 09:21:00.00000037,28.1,2009-12-29 09:21:00 UTC,-73.984918,40.758758,-73.873383,40.774368,1,2009-12-29,09:21:00,...,9.555103,11.133425,9,8,8.1,28.9,32.5,8765,32.27,33.83
2,2010-02-13 12:34:29.0000001,3.3,2010-02-13 12:34:29 UTC,-73.968866,40.754251,-73.974596,40.751740,1,2010-02-13,12:34:29,...,0.557775,0.762099,0,0,7.5,7.7,5.5,92983,5.50,5.70
3,2012-08-03 14:27:00.000000205,7.7,2012-08-03 14:27:00 UTC,-73.976125,40.760132,-73.984120,40.740300,2,2012-08-03,14:27:00,...,2.306489,2.879497,22,4,7.7,7.5,6.1,88510,6.10,6.50
4,2015-05-19 22:33:02.0000009,18.0,2015-05-19 22:33:02 UTC,-73.977867,40.749809,-73.948914,40.808022,1,2015-05-19,22:33:02,...,6.918961,8.914686,22,3,7.7,10.0,14.5,5795,14.50,14.10


In [ ]:
# # Test - Train Split
# train=rd.sample(range(len(df)),round(0.8*len(df)))
# test=list(set(range(len(df)))-set(train))

In [33]:
# dftrain=df.iloc[train,:]
# dftest=df.iloc[test,:]
df.to_csv('final_train2.csv')
dftest.to_csv('final_test2.csv')
print (len(df),len(dftest))

4000000 9914


In [ ]:
cols=['passenger_count', 'month', 'year', 'weekday', 'hour',
       'week_of_month', 'processed_hour_num_of_records', 'processed_hour_fare',
       'Distance', 'Manhattan_Distance', 'Pickup_clusters', 'Dropoff_clusters',
       'processed_pickup_fare', 'processed_dropoff_fare',
       'processed_route_fare', 'processed_route_rank',
       'processed_route_people_fare', 'processed_route_hour_people_fare']
labelcol='fare_amount'

In [ ]:
reg1=LinearRegression()
reg1.fit(dftrain[cols].values,dftrain[labelcol].values)
np.sqrt(mean_squared_error(dftest[labelcol].values,reg1.predict(dftest[cols].values)))

In [ ]:
reg2=RandomForestRegressor(n_estimators=100)
reg2.fit(dftrain[cols].values,dftrain[labelcol].values)
np.sqrt(mean_squared_error(dftest[labelcol].values,reg2.predict(dftest[cols].values)))

In [ ]:
reg3=RandomForestRegressor(n_estimators=50)
reg3.fit(dftrain[cols].values,dftrain[labelcol].values)
np.sqrt(mean_squared_error(dftest[labelcol].values,reg3.predict(dftest[cols].values)))

In [ ]:
reg4=RandomForestRegressor(n_estimators=200)
reg4.fit(dftrain[cols].values,dftrain[labelcol].values)
np.sqrt(mean_squared_error(dftest[labelcol].values,reg4.predict(dftest[cols].values)))

In [ ]:
reg5=SVR()
reg5.fit(dftrain[cols].values,dftrain[labelcol].values)
np.sqrt(mean_squared_error(dftest[labelcol].values,reg5.predict(dftest[cols].values)))

In [ ]:
np.sqrt(mean_squared_error(dftest[labelcol].values,reg5.predict(dftest[cols].values)))

In [ ]:
reg4.feature_importances_

In [ ]:
test=pd.read_csv('test.csv')
len(test)